In [23]:
from bs4 import BeautifulSoup
from math import log
import pandas as pd
from time import sleep

In [24]:
df = pd.read_csv('words.txt', sep=' ', header=None)
df[0].tolist()
all_guesses = [x for x in df[0].tolist() if len(str(x)) == 5]

In [25]:
all_guesses = [x.lower() for x in all_guesses if x.isalpha()]
# len(all_guesses)

In [26]:
def init_score(guess):
    for i, letter in enumerate(guess):
        if letter == solution[i]:
            guess_scores[i] = g
            print(g)
        elif letter in solution:
            guess_scores[i] = y
            print(y)
        else:
            misses.append(letter)
            guess_scores[i] = m
            print(m)
    return guess_scores, misses
# print(misses)
# print(guess_scores)

In [27]:
def remove_words(options, misses, guess, guess_scores):
    removes = []
    # print(f"length of options before anything: {len(options)}\n")

    # removing any words with letters from the misses list

    for word in options:
        for letter in misses:
            if letter in word:
                # print(word, letter)
                removes.append(word)
                break

    options_2 = [x for x in options if x not in removes]
    # print(f"length of options after misses: {len(options_2)}\n")

    # removing any words with letters that don't match the green letter at its placement

    for word in options_2:
        for i, score in enumerate(guess_scores):
            if score == 2:
                true_letter = guess[i]
                if word[i] != true_letter:
                    # print(word, guess, true_letter)
                    removes.append(word)
                    break

    options_3 = [x for x in options_2 if x not in removes]
    # print(f"length of options after misses and green letter: {len(options_3)}\n")

    # removing any words that don't contain the yellow letters

    yellow_letters = []
    yl = {}                                
    for i, digit in enumerate(guess_scores):
        if digit == 1:
            if guess[i] in yellow_letters:
                yl[guess[i]].append(i)
            else:
                yl[guess[i]] = [i]
            yellow_letters.append(guess[i])
    
    for word in options_3:
        for letter in yellow_letters:
            if letter not in word:
                removes.append(word)
                break

    options_4 = [x for x in options_3 if x not in removes]

    # print(f"length of options after misses, green letter, and yellow letter: {len(options_4)}\n")

    for word in options_4: # TODO: THIS IS ANOTHER AREA THAT NEEDS WORK
        for letter in yl:
            for index in yl[letter]:    
                if word[index] == letter:
                    removes.append(word)
                    break

    options_5 = [x for x in options_4 if x not in removes]

    # print(f"length of options after misses, green letter, yellow letter, and yellow letter placement elimination: {len(options_5)}\n")

    for word in options_5:
        for letter in yl:
            if word.count(letter) < len(yl[letter]):
                removes.append(word)
                break
            
    options_6 = [x for x in options_5 if x not in removes]

    ml = []
    multiple_letters = {False: ''}

    for letter in yl:
        if len(yl[letter]) > 1:
            val = {letter: len(yl[letter])}
            ml.append(val)
    for letter in yl:
        if len(yl[letter]) > 1:
            multiple_letters = {True: ml}
            break
        else:
            multiple_letters = {False: ''}
    
    # print(f"length of options after misses, green letter, yellow letter, yellow letter placement, and yellow letter count: {len(options_6)}\n")

    return options_6, multiple_letters


In [28]:
def filter_by_score(options, guess, guess_scores):
    scores = {}
    for word in options:
        score = 0
        for i, letter in enumerate(word):
            if guess_scores[i] == 0:
                continue
            elif guess_scores[i] == 1:
                if guess[i] in word:
                    score += guess_scores[i]
            elif guess_scores[i] == 2:
                if letter == guess[i]:
                    score += guess_scores[i]
        scores[word] = score

    best_score = max(scores.values())
    return_options = []
    for word in options:
        if scores[word] == best_score:
            return_options.append(word)
    # print(return_options)
    return return_options
        

In [29]:
def get_letter_freq():

    s = '''Letter	Count	 	Letter	Frequency
    E	21912	 	E	12.02
    T	16587	 	T	9.10
    A	14810	 	A	8.12
    O	14003	 	O	7.68
    I	13318	 	I	7.31
    N	12666	 	N	6.95
    S	11450	 	S	6.28
    R	10977	 	R	6.02
    H	10795	 	H	5.92
    D	7874	 	D	4.32
    L	7253	 	L	3.98
    U	5246	 	U	2.88
    C	4943	 	C	2.71
    M	4761	 	M	2.61
    F	4200	 	F	2.30
    Y	3853	 	Y	2.11
    W	3819	 	W	2.09
    G	3693	 	G	2.03
    P	3316	 	P	1.82
    B	2715	 	B	1.49
    V	2019	 	V	1.11
    K	1257	 	K	0.69
    X	315	 	X	0.17
    Q	205	 	Q	0.11
    J	188	 	J	0.10
    Z	128	 	Z	0.07
    '''

    raw_df = pd.DataFrame([x.split('\t') for x in s.split('\n') if x != ''], columns=['letter', 'count', '','letter2', 'frequency'])
    raw_df.drop(columns=['', 'count','letter2'], inplace=True)
    raw_df.drop(index=0, inplace=True)
    df = raw_df.set_index('letter')
    d = df.to_dict()['frequency']
    keys = [x.strip().lower() for x in d]
    return {key:float(f) for (key,f) in zip(keys, d.values()) if key !=''}

# get_letter_freq()

In [30]:
# calculate confidence based strictly on how many green and yellow letters there are

def green_yellow_confidence(green_count, yellow_count, pool_size, total_letters=5, weight_green=.7, weight_yellow=.3):
    # Avoid division by zero
    if pool_size == 0:
        return 0

    # TODO: add a factor for the remaining words in the alphabet
    # TODO: add a factor for the popularity of the letter (z should be lower in rank than s)

    score = ((green_count * weight_green + yellow_count * weight_yellow) / total_letters) * (1 / pool_size)
    return score

# print(green_yellow_confidence(1, 0, 10))
# print(green_yellow_confidence(0, 5, 10))


In [31]:
# calculate the confidence that the word is the solution

def build_confidence_dict(words, multiple_letters):
    confidence_dict = {}
    letter_freq = get_letter_freq()

    # compiling a dictionary of each word and the sum of frequency score of each of its letters

    freq_scores = {}
    for word in words:
        freq_scores[word] = 0
        for letter in word:
            freq_scores[word] += letter_freq[letter]

    # adding the frequency score to the confidence score

    for word in freq_scores:
        confidence_dict[word] = (freq_scores[word] * 0.01)

    # adding a penalty for words with multiple letters not accounted for

    if not list(multiple_letters.keys())[0]:
        for word in confidence_dict:
            if len(set(word)) < len(word):
                confidence_dict[word] -= (0.05 * (len(word)-len(set(word))))
    else:
        for word in confidence_dict:
            for letter in word:
                if word.count(letter) > 1:
                    if letter not in list(multiple_letters[True][0].keys()):
                        confidence_dict[word] -= 0.05

    for key in confidence_dict:
        confidence_dict[key] = round(confidence_dict[key], 3)
    
    return dict(sorted(confidence_dict.items(), key=lambda item: item[1], reverse=True))
    
# build_confidence_dict(['child','mitch', 'clive', 'livid', 'pitch'], {False: ''})

In [32]:
# response = get('https://www.wordunscrambler.net/word-list/wordle-word-list')
# bs = BeautifulSoup(response.text, 'html.parser')
# all_a_html = bs.find_all('a')
# all_a = [a.text for a in all_a_html]
# all_words = [word for word in all_a if len(word) == 5]
# pool = [word for word in all_words if word not in past_words]

In [33]:
# we need to adjust ALL the weights to not be arbitrary

# Jocelyn guessed eaten when it was taken and we got an error]\

# input that a second e gave me a 0 score and it messed it up

# guessed state and it removed options with t because the second t was a 0 score

In [34]:
def compare_lists(popularity, confidence):
    weight_commonality = 0.55
    weight_confidence = 0.45
    combined_scores = {}
    for i, word in enumerate(popularity):
        rank = i + 1  # Rank starting from 1
        normalized_rank = 1 / rank  # Normalizing rank

        # Combined score calculation
        combined_score = (normalized_rank * weight_commonality) + (confidence[word] * weight_confidence)
        combined_scores[word] = round(combined_score,4)

    # Sort words by combined score
    best_guesses = sorted(combined_scores, key=combined_scores.get, reverse=True)
    return dict(sorted(combined_scores.items(), key=lambda item: item[1], reverse=True))

# compare_lists(['reply'],{'reply': 0.4829})

In [43]:
# this is not working too well right now - I keep getting words like alert which shares too many letters with serai. 


def get_word_information(options, letters_seen_before):
    letter_info = {}
    word_info = {}
    alphabet = [chr(x) for x in range(97, 123)] # getting the alphabet
    letters_in_every_option = []

    for letter in alphabet:

        # counting the number of words the letter IS NOT present in the list of options
        times_letter_absent = len([x for x in options if letter not in x])

        # if the letter more than 0 but less than the total amount we want to use the info gain formula
        if times_letter_absent > 0 and times_letter_absent < len(options):
            letter_information = log(1/(times_letter_absent/len(options)), 2)
            if letter_information == 0:
                letters_in_every_option.append(letter)

            # if we have already encountered the letter in previous guesses, we want to penalize it
            if letter in letters_seen_before:
                letter_information = 0
        else:

            # if the letter doesn't occur in any of the options or occurs in ALL the options we give it a 0
            letter_information = 0

        letter_info[letter] = letter_information

    # organizing by information gain amount
    letter_info = {k: v for k, v in sorted(letter_info.items(), key=lambda item: item[1])}


    for word in options:
        sum = 0.0
        if len(set(word)) < len(word):

            # arbitrary amount set for now
            sum -= 0.3

        prev_letters = []
        for letter in word:

            # if the letter is in every option, we want to penalize 
            if letter in letters_in_every_option:

                # arbitrary amount set for now
                sum -= 0.3

            # adding the information gain of each letter in the word
            # IF it hasn't been seen before
            if letter not in prev_letters:
                sum += letter_info[letter]
                prev_letters.append(letter)
        word_info[word] = sum

        # sorting by infoqrmation gain amount
        word_info = {k: v for k, v in sorted(word_info.items(), key=lambda item: item[1])}
        word_info = dict(sorted(word_info.items(), key=lambda item: item[1], reverse=True))

    return word_info

get_word_information(all_options, [])


{'raise': 3.13294090648177,
 'arise': 3.13294090648177,
 'laser': 3.13294090648177,
 'arose': 3.1142696299160164,
 'stare': 3.101483474960173,
 'alert': 3.0971401850539237,
 'later': 3.0971401850539233,
 'saner': 3.048712999705005,
 'renal': 3.044369709798756,
 'aisle': 2.9997676362305685,
 'least': 2.9683102047089713,
 'steal': 2.9683102047089713,
 'early': 2.948110407922961,
 'relay': 2.948110407922961,
 'teary': 2.916652976401364,
 'aider': 2.9016690630327644,
 'share': 2.8888225916414982,
 'shear': 2.8888225916414982,
 'spare': 2.8827326921771244,
 'parse': 2.8827326921771244,
 'spear': 2.8827326921771244,
 'leant': 2.8797390080259566,
 'pearl': 2.8783894022708747,
 'paler': 2.8783894022708747,
 'tread': 2.870211631511167,
 'realm': 2.847129786792271,
 'risen': 2.8423733168918317,
 'rinse': 2.8423733168918317,
 'siren': 2.8423733168918317,
 'liner': 2.838030026985582,
 'large': 2.8364650520940518,
 'glare': 2.8364650520940518,
 'lager': 2.8364650520940518,
 'baler': 2.8247070812178

In [36]:
def information_gain_words(letters_to_remove, words):
    ret = []
    for word in words:
        valid = True
        for letter in letters_to_remove:
            if letter in word:
                valid = False
                break
        if valid:
            ret.append(word)
    return ret

# information_gain_words(['a','v','i'], ['abc', 'def', 'ghi', 'jkl', 'mno', 'pqr', 'stu', 'vwx', 'yz'])

In [44]:
all_options = pd.read_csv('options.csv')['word'].tolist()
options = all_options
keep_guessing = True
count = 0
correct_word = ''
misses = []
all_letters_so_far = []

while keep_guessing:
    count += 1
    guess = input('What did we guess?')
    if guess == '':
        break
    guess_color_sequence = input('What was the color sequence?\n\'g\' for green\n\'y\' for yellow\nand \'b\' for bad guess (kinda hard when two of the colors - green and grey - both start with the same letter).')
    guess_scores = []
    

    for i, color in enumerate(guess_color_sequence):
        if color == 'g':
            color = 2
        elif color == 'y':
            color = 1
        else:
            color = 0
            misses.append(guess[i])
        guess_scores.append(color)

    all_letters_so_far += list(guess)

    # mistake here: we shouldn't remove words with letters we already guess yet. Example: match, catch, batch, watch, latch

    info_words_sorted = get_word_information(all_options, all_letters_so_far)
    
    o, ml = remove_words(options, misses, guess, guess_scores)
    d = build_confidence_dict(o, ml)
    dinfo = {k: v for k, v in sorted(info_words_sorted.items(), key=lambda item: item[1])}
    
    print(f'\ntop final option(s) ranked by word popularity (max 10): {o[:10]}\n')
    print(f'Top guesses that will narrow down the options the most:{list(reversed(dinfo.keys()))}\n')

    if input('Take a look at the options below and input your guess. If we guessed correctly, type "correct", otherwise, hit enter') == 'correct':
        count +=1
        keep_guessing = False
        correct_word = input(f'\nNice job! We got it in {count} guesses! What was the correct word?\n')
        continue
    
    options = o


top final option(s) ranked by word popularity (max 10): ['order', 'never', 'offer', 'cover', 'entry', 'lower', 'owner', 'buyer', 'upper', 'creek']

Top guesses that will narrow down the options the most:['clout', 'hotly', 'notch', 'blunt', 'cloud', 'nobly', 'tonal', 'talon', 'blond', 'pouty', 'lunch', 'thong', 'moldy', 'octal', 'clung', 'godly', 'mouth', 'mogul', 'junto', 'clone', 'mound', 'botch', 'loath', 'bound', 'flown', 'stony', 'olden', 'motel', 'bloat', 'wound', 'lousy', 'pouch', 'truly', 'block', 'north', 'mulch', 'downy', 'melon', 'clump', 'lumpy', 'court', 'gulch', 'flung', 'along', 'login', 'lingo', 'towel', 'glint', 'cough', 'ingot', 'tango', 'tonga', 'punch', 'butch', 'quoth', 'tulip', 'ounce', 'felon', 'shout', 'south', 'latch', 'tunic', 'aunty', 'unity', 'dough', 'flint', 'flunk', 'spout', 'crony', 'munch', 'lymph', 'torch', 'often', 'optic', 'loamy', 'boule', 'novel', 'guilt', 'tumor', 'token', 'tubal', 'gusto', 'daunt', 'plumb', 'synod', 'turbo', 'comfy', 'extol', 'bu